# PKGS

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from IPython.display import display

pd.set_option('display.float_format', '{:,.2f}'.format)

# VaR/CVaR Forex

In [2]:
def VaR_CVaR_Fx(data, positions, conf, long):
     port = data * positions 
     total = port.sum(axis=1)
     rt = total.pct_change().dropna()

     var = np.percentile(rt,100-conf) if long else np.percentile(rt,conf)
     cvar = rt[rt < var].mean() if long else rt[rt > var].mean()

     var_m = var * total.iloc[-1]
     cvar_m = cvar * total.iloc[-1]

     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var, cvar], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [3]:
Fx = ['CHFMXN=X', 'MXN=X']

data = yf.download(Fx, start='2020-01-01', end='2024-12-02')['Close'][Fx] #No puedo descargar el ajustado

positions = [5300000, 7100000] # En mismo orden que Fx

long = True #True o False

NC = 99 #[0, 100]

VaR_CVaR_Fx(data, positions, NC, long)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


,Métrica,Porcentual (%),Monetario ($)
0,VaR,-0.02,"-4,918,876.94"
1,CVaR,-0.02,"-6,351,375.41"


# VaR/CVaR Stocks

In [4]:
def VaR_CVaR_Stocks(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]
     weights = [valor_posicion.iloc[i]/valor_posicion.sum() for i in range(len(n_stocks))]  
     rt_assets = data.pct_change().dropna()
     rt_port = rt_assets.mul(weights).sum(axis=1)

     var = np.percentile(rt_port,100-conf) if long else np.percentile(rt_port,conf)
     cvar = rt_port[rt_port < var].mean() if long else rt_port[rt_port > var].mean()
          
     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()
     
     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [5]:
Stocks = ['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA']

data = yf.download(Stocks, start="2020-01-01", end="2023-01-01")['Close'][Stocks] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] # En mismo orden que Stocks

long = True #True o False

conf = 95 #[0, 100]

VaR_CVaR_Stocks(data, positions, conf, long)

[*********************100%***********************]  5 of 5 completed


,Métrica,Porcentual (%),Monetario ($)
0,VaR,-2.88,"-33,073.79"
1,CVaR,-4.63,"-53,131.46"


# Rebalanceo

In [6]:
def rebalanceo(initial_weights, optimal_weights):
     
     df = pd.DataFrame({"Initial Weights": initial_weights*100, "Optimal Weights": optimal_weights*100, "Stocks difference": np.floor((optimal_weights - initial_weights) * portfolio_value / data.iloc[-1])})
     df['Buy/Sell'] = df['Stocks difference'].apply(lambda x: 'Buy' if x > 0 else ('Sell' if x < 0 else 'NA')) if long else df['Stocks difference'].apply(lambda x: 'Sell' if x > 0 else ('Buy' if x < 0 else 'NA'))
     
     return df